# What is Docker?

Docker makes development efficient and predictable
Docker takes away repetitive, mundane configuration tasks and is used throughout the development lifecycle for fast, easy and portable application development - desktop and cloud. Docker’s comprehensive end to end platform includes UIs, CLIs, APIs and security that are engineered to work together across the entire application delivery lifecycle.


[Official docker page](https://www.docker.com/)

# How does it work?

![docker containers vs VMs](https://lh6.googleusercontent.com/JLzilHI8wTeXfpclTthVU8Tkg4F_pWBbFwf8lsgHycscIAcK8fslXQlW06lb5lYnfEK-eJXHvvSFNbnANY5A4O6js7KY1ic-Y1jvt-UbTB3aHzq7AOrv0CSotz8vpK5K4Xpj0BMH)

# Why Docker?


1. Easily change new version - compatibility check
2. Image on your machine == image on your colleagues machine
3. Minimal images for faster shipping and storing
4. Everything is isolated - modular and secure

![No more excuses?](https://miro.medium.com/max/800/1*DPS45Tufsih-zED3To4k6g.jpeg)


# My Plan is to cover minimal knowledge in order to help you create dev env

# I will not go into the architecture!

Reasons - not really sure about everything and whole presentation is based on my playing around experience. Don't want to make a fool out of myself saying something not entirely true.

![Let's get down to business](https://media.tenor.com/images/81073b6af5e0988f351951444ad27e53/raw)

We have a neat command named ps. It lists all the running processes

In [1]:
!ps

  PID TTY           TIME CMD
67106 ttys001    0:01.50 -zsh
68107 ttys001    0:02.18 /Library/Frameworks/Python.framework/Versions/3.7/Reso


We have analogy to that in docker: **docker ps**

In [2]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


We have 0 docker processes running

# How we can run some docker container on our machine?

We can used **docker run**

![docker run](https://www.devteam.space/wp-content/uploads/2017/03/container_explainer.png)


# Let's try and run container with postgres DB on our machine.

## Step 1

Go to google and search for **docker postgres** and choose the [hub.docker.com one](https://hub.docker.com/_/postgres)

## Step 2

Figure out which version you are going to use. at the moment I'm preparing this we have something like:

13.2, 13, latest


13.2-alpine, 13-alpine, alpine


12.6, 12


12.6-alpine, 12-alpine


Options we have:
* specific version
* latest
* specific version - alpine

What's this **alpine**?

Alpine is minimal viable image of the container. Bare bones as you might say. If you are app/code is working with alpine - go for it. 

**Benefits of alpine**:
* small image - faster to pull and run
* if something is missing - you can easily add with additional code in docker file (will be covered in later parts) 

## Step 3

**docker run postgres:VERSION_YOU_CHOSE**

[docker run](https://docs.docker.com/engine/reference/commandline/run/)


In [9]:
!docker run postgres:13-alpine


Error: Database is uninitialized and superuser password is not specified.
       You must specify POSTGRES_PASSWORD to a non-empty value for the
       superuser. For example, "-e POSTGRES_PASSWORD=password" on "docker run".

       You may also use "POSTGRES_HOST_AUTH_METHOD=trust" to allow all
       connections without a password. This is *not* recommended.

       See PostgreSQL documentation about "trust":
       https://www.postgresql.org/docs/current/auth-trust.html


**Oh... I need to pass the password...**

Also in order to don't stop Jupyter notebook we need to detach the process.

Detaching: **-d**



In [11]:
!docker run -e POSTGRES_PASSWORD=mysecretpassword -d postgres:13-alpine 

165f736ad9a6b4572743d8ff6169239297026fa9898518e6f3c139804fbcebaa


# Verification of process

In [12]:
!docker ps

CONTAINER ID   IMAGE                COMMAND                  CREATED        STATUS                  PORTS      NAMES
165f736ad9a6   postgres:13-alpine   "docker-entrypoint.s…"   1 second ago   Up Less than a second   5432/tcp   eager_galileo


## If you'd try to connect with some other tool to check DB you won't see it..

## In order to see it we need to map that port for us to access it


Port exposure:

**docker run -p HOST_PORT:CONTAINER_PORT DOCKER_IMAGE**


In [13]:
!docker run -p 5400:5432 -e POSTGRES_PASSWORD=mysecretpassword -d postgres:13-alpine 

aa036162846937aabccb69c700bfd37715d05a065dcbc1ec7e8145559e6e9583


![Let's celebrate](https://media.giphy.com/media/lMameLIF8voLu8HxWV/giphy.gif)

ok, enough of celebrations!

You probably have a question about docker run result we got back, right?

**aa036162846937aabccb69c700bfd37715d05a065dcbc1ec7e8145559e6e9583**

What is it?

this is your full docker container id. If you check closely to **docker ps** result: **aa036162846937a** it's a match to the first part of the full container id.

in case you want to run your docker container again you can use shorter substring of it and docker will map it.

let's stop the container using **docker stop CONTAINER_ID**

__BTW: There is an option to **kill** your container. Difference between them: Stop - shutdown gracefully.__

In [18]:
!docker stop aa036162846937aab 

aa036162846937aab


# Do I have to each time to use the long docker run command for same container?

Answer: No!

In [19]:
!docker start aa036162846937aab 

aa036162846937aab


In [20]:
!docker ps

CONTAINER ID   IMAGE                COMMAND                  CREATED          STATUS                  PORTS                    NAMES
aa0361628469   postgres:13-alpine   "docker-entrypoint.s…"   47 seconds ago   Up Less than a second   0.0.0.0:5400->5432/tcp   suspicious_davinci


# But wait, how do I know my docker container id if I need it tomorrow?
![confusion](https://i.kym-cdn.com/entries/icons/original/000/018/489/nick-young-confused-face-300x256-nqlyaa.jpg)


# No worries, there is a solution!

Let's stop the container again.

In [21]:
!docker stop aa036162846937aab

aa036162846937aab


docker ps has this option to show past processes/containers running with passing more parameters [link to docs](https://docs.docker.com/engine/reference/commandline/ps/)

In [22]:
!docker ps -all

CONTAINER ID   IMAGE                COMMAND                  CREATED              STATUS                     PORTS     NAMES
aa0361628469   postgres:13-alpine   "docker-entrypoint.s…"   About a minute ago   Exited (0) 2 seconds ago             suspicious_davinci


BAM! now if something changes you can always go back and get it whenever you need it.


**Is there an easier way to create those containers without passing params and etc?**

**Answer to that: YES**

it's Dockerfile





# Dockerfile

Dockerfile is a config specific for docker and containing all environment variables related to your container.

## This code is already in the Dockerfile inside this directory
FROM postgres:13-alpine


ENV POSTGRES_PASSWORD=myverybadpassword



## Verify that you see this:

In [23]:
!cat Dockerfile

FROM postgres:13-alpine
ENV POSTGRES_PASSWORD=myverybadpassword

in order to create our own local image we need to build Dockerfile

for this we're going to use **docker build .** 

Dot here means we're build a Dockerfile inside this directory. If you want you can move your Dockerfile to different directory and do **docker build ./DIRECTORY/.**

In [29]:
!docker build .

Sending build context to Docker daemon  58.88kB
Step 1/2 : FROM postgres:13-alpine
 ---> bfc401b315be
Step 2/2 : ENV POSTGRES_PASSWORD=myverybadpassword
 ---> Using cache
 ---> af178fb8a3eb
Successfully built af178fb8a3eb


We have our own image id now: **af178fb8a3eb**

let's run it with same port mapping

In [33]:
!docker run -p 5400:5432 -d af178fb8a3eb

4604c463e7e6b359993130eb6b66fd055c8f5adb69d7d44bb37bb246c088a939


In [34]:
!docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED         STATUS         PORTS                    NAMES
4604c463e7e6   af178fb8a3eb   "docker-entrypoint.s…"   5 seconds ago   Up 4 seconds   0.0.0.0:5400->5432/tcp   awesome_shaw


In [35]:
!docker stop 4604c463e7e6

4604c463e7e6


# One more thing...

We can add tags to our docker build in order to access the dockerfile in something like tomasp_special_env:latest

all you have to do is to add a tag with **-t** parameter

In [36]:
!docker build -t tomas_postgres_local:latest .

Sending build context to Docker daemon  58.88kB
Step 1/2 : FROM postgres:13-alpine
 ---> bfc401b315be
Step 2/2 : ENV POSTGRES_PASSWORD=myverybadpassword
 ---> Using cache
 ---> af178fb8a3eb
Successfully built af178fb8a3eb
Successfully tagged tomas_postgres_local:latest


In [37]:
!docker run -d tomasp_postgres_version:latest

4a2f48d620b6296e91def1c5db5abc75a4181fa1d2b57d79121b65cdf46ebeab


In [38]:
!docker ps

CONTAINER ID   IMAGE                            COMMAND                  CREATED         STATUS        PORTS      NAMES
4a2f48d620b6   tomasp_postgres_version:latest   "docker-entrypoint.s…"   2 seconds ago   Up 1 second   5432/tcp   pensive_tharp


# Also let's run python docker image

In [32]:
!docker run python:3.7-alpine

Unable to find image 'python:3.7-alpine' locally
3.7-alpine: Pulling from library/python

57a56b15: Already exists 
5f2e27e9: Already exists 
aa61232f: Already exists 
eb02ec5c: Already exists 
9520c775: Already exists Digest: sha256:9e53915453001b112e3d88a4b8ebcf2229ff943c202b5335dccb71917d49eae5
Status: Downloaded newer image for python:3.7-alpine


# But I hope you already see a flaw?

In order to setup all working environment it's a bit annoying to run everything one by one. What if they have dependencies? Is there a solution?

**Answer: YES!**

## Docker compose

It's a different configuration file which allows you to quickly add multiple Dockerfiles or different images and do whatever you specify with them. In order to run your images you just need to docker compose and run the image id 

### Structure:
**docker-compose.yml** file ([link to official page](https://docs.docker.com/compose/)):

```yml
version: "3.8"
services:
  app :
    image: "image_you_will_use"
    container_name: "container_name"
    ports:
      - 8080:8080
  db:
    build: ./database/
```
* version means compose file format (has to work with your docker engine)  [link to docs](https://docs.docker.com/compose/compose-file/)
* app from image and pass necessary information in docker-compose file
* db we build from dockerfile inside the database directory

**We're going to use -d to run it as detached**

In order to build/compose it to a docker image run docker-compose either in the directory you are in with:
```bash
docker-compose up -d --build
```

or pass the file:

```bash
docker-compose -f "PATH/YOU/HAVE/PUT/DOCKERCOMPOSE/FILE/docker-compose.yml" up -d --build
```

[link to docker compose docs](https://docs.docker.com/compose/reference/overview/)

# All in all - Docker FTW.

It makes development life easier, create image once, run your apps on top of it and see if it works.

My personal reasons why:
* Need an isolated ENV for benchmarking/testing etc
* Testing version upgrade if something breaks
* If you messed up your local env and you're just lazy to clean it